This notebook contains the code and all my understandings and everything about magic. This is somehow my first serious work in machine learning.

In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import time

In [2]:
# TODO: make these constants arguments
n_route = 228
n_hist = 12
n_pred = 9
batch_size = 50
epoch = 50
# parser.add_argument('--save', type=int, default=10)
ks = 3  # spatial convolution kernel size
kt = 3  # temporal convolution kernel size
lr = 1e-3
opt = torch.optim.RMSprop
# parser.add_argument('--graph', type=str, default='default')
inf_mode = 'merge'
rate = 0.1

The model has two spatio temporal blocks and each block has two gated temporal convolutional layers and one spatial convolutional layer in between so we have totaly six layers and we should determine the size of channel for these layers

In [3]:
# blocks = [[1, 32, 64], [64, 32, 128]]
blocks = [[1, 32, 64]]

We should have a file which defines the graph we want to work on.

In [4]:
W = pd.read_csv("PeMSD7_W_228.csv", header=None)
W_tensor = torch.tensor(W.values)
W_tensor.shape

torch.Size([228, 228])

Equation 10 on original papaer

In [5]:
def weight_matrix(W, sigma2=0.1, epsilon=0.5, scaling=True):
    '''
    Weight matrix function.
    :param W: tensor, the adjacency matrix of the graph.
    :param sigma2: float, scalar of matrix W.
    :param epsilon: float, thresholds to control the sparsity of matrix W.
    :param scaling: bool, whether applies numerical scaling on W.
    :return: np.ndarray, [n_route, n_route].
    '''
    # check whether W is a 0/1 matrix.
    if set(torch.unique(W)) == {0, 1}:
        print('The input graph is a 0/1 matrix; set "scaling" to False.')
        scaling = False

    if scaling:
        n = W.shape[0]
        W = W / 10000. # WHY
        W2, W_mask = W * W, np.ones([n, n]) - np.identity(n)
        # refer to Eq.10
        a = torch.exp(-W2 / sigma2)
        a[a >= epsilon] = 0
        
        return a * W_mask
    
    return W

In [6]:
tmp = weight_matrix(W_tensor)
print(torch.unique(tmp))

tensor([0.0000e+00, 1.4279e-89, 1.0224e-88,  ..., 4.9829e-01, 4.9897e-01,
        4.9999e-01], dtype=torch.float64)


In the below cell I want to calculate normalized laplacian.<br/>
I calculate it using \(L=I - D^(-1/2) W D^(-1/2)\)

In [34]:
def scaled_laplacian(W):
    '''
    Scaled graph Laplacian function.
    :param W: np.ndarray, [n_route, n_route], weighted adjacency matrix of G.
    :return: np.matrix, [n_route, n_route].
    '''
    # d ->  diagonal degree matrix
    n, d = np.shape(W)[0], torch.sum(W, axis=1)
    I = torch.eye(n)
    
    # d^(-1/2)
    d = torch.pow(d, -1/2)
    # make d diagonal
    d = torch.diag(d)

    dw = torch.matmul(d, W)
    dwd = torch.matmul(dw, d)
    
    L = I - dwd
    
    # lambda_max \approx 2.0, the largest eigenvalues of L.
    lambdas, _ = torch.eig(L)
    lambdas = lambdas[:, 0]
    lambda_max = torch.max(lambdas)

    return 2 * L / lambda_max - I

In [35]:
L = scaled_laplacian(torch.rand(4, 4))

d
tensor([1.3467, 2.6622, 2.0579, 1.9413])
d 1/2
tensor([[0.8617, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6129, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6971, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7177]])
W
tensor([[0.0210, 0.3091, 0.4645, 0.5522],
        [0.8119, 0.3450, 0.6997, 0.8057],
        [0.5119, 0.3486, 0.3314, 0.8661],
        [0.1804, 0.7242, 0.1198, 0.9170]])
dw
tensor([[0.0181, 0.2663, 0.4003, 0.4758],
        [0.4976, 0.2114, 0.4289, 0.4938],
        [0.3568, 0.2430, 0.2310, 0.6037],
        [0.1295, 0.5197, 0.0860, 0.6581]])
dwd
tensor([[0.0156, 0.1632, 0.2790, 0.3415],
        [0.4288, 0.1296, 0.2989, 0.3544],
        [0.3075, 0.1489, 0.1610, 0.4333],
        [0.1116, 0.3185, 0.0599, 0.4723]])
return
tensor([[ 0.5914, -0.2639, -0.4511, -0.5521],
        [-0.6932,  0.4071, -0.4833, -0.5729],
        [-0.4970, -0.2408,  0.3563, -0.7005],
        [-0.1804, -0.5150, -0.0969, -0.1470]])


In [36]:
T = torch.tensor([[0,2,3], [2, 0, 5], [3, 5, 0]], dtype=float)
print(scaled_laplacian(T))

d
tensor([5., 7., 8.], dtype=torch.float64)
d 1/2
tensor([[0.4472, 0.0000, 0.0000],
        [0.0000, 0.3780, 0.0000],
        [0.0000, 0.0000, 0.3536]], dtype=torch.float64)
W
tensor([[0., 2., 3.],
        [2., 0., 5.],
        [3., 5., 0.]], dtype=torch.float64)
dw
tensor([[0.0000, 0.8944, 1.3416],
        [0.7559, 0.0000, 1.8898],
        [1.0607, 1.7678, 0.0000]], dtype=torch.float64)
dwd
tensor([[0.0000, 0.3381, 0.4743],
        [0.3381, 0.0000, 0.6682],
        [0.4743, 0.6682, 0.0000]], dtype=torch.float64)
return
tensor([[ 0.1841, -0.4003, -0.5617],
        [-0.4003,  0.1841, -0.7912],
        [-0.5617, -0.7912,  0.1841]], dtype=torch.float64)
tensor([[ 0.1841, -0.4003, -0.5617],
        [-0.4003,  0.1841, -0.7912],
        [-0.5617, -0.7912,  0.1841]], dtype=torch.float64)


**Chebyshev polynomials approximation**

In [10]:
def cheb_poly_approx(L, Ks, n):
    '''
    Chebyshev polynomials approximation function.
    :param L: np.matrix, [n_route, n_route], graph Laplacian.
    :param Ks: int, kernel size of spatial convolution.
    :param n: int, number of routes / size of graph.
    :return: np.ndarray, [n_route, Ks*n_route].
    '''
    L0, L1 = np.identity(n), np.copy(L)

    if Ks > 1:
        L_list = [np.copy(L0), np.copy(L1)]
        for i in range(Ks - 2):
            Ln = 2 * L * L1 - L0
            L_list.append(np.copy(Ln))
            L0, L1 = np.copy(L1), np.copy(Ln)
        # L_lsit [Ks, n*n], Lk [n, Ks*n]
        return np.concatenate(L_list, axis=-1)
    elif Ks == 1:
        return np.asarray(L0)
    else:
        raise ValueError(f'ERROR: the size of spatial kernel must be greater than 1, but received "{Ks}".')

In [11]:
# Alternative approximation method: 1st approx - first_approx(W, n).
Lk = cheb_poly_approx(L, ks, n_route)

In [12]:
import pandas as pd

data_file = pd.read_csv("PeMSD7_V_228.csv", header=None)
data_file_tensor = torch.tensor(data_file.values)
data_file_tensor

tensor([[71.1000, 66.0000, 64.6000,  ..., 69.3000, 67.7000, 68.9000],
        [68.1000, 66.8000, 61.7000,  ..., 67.7000, 68.8000, 68.8000],
        [68.0000, 64.3000, 66.6000,  ..., 70.2000, 69.1000, 68.7000],
        ...,
        [68.9000, 37.9000, 68.8000,  ..., 68.7000, 67.5000, 19.7000],
        [69.2000, 37.8000, 68.7000,  ..., 67.7000, 67.4000, 19.5000],
        [68.6000, 52.9000, 68.6000,  ..., 68.1000, 68.5000, 19.1000]],
       dtype=torch.float64)

In [13]:
n_train, n_val, n_test = 34, 5, 5

In [14]:
def seq_gen(len_seq, data_seq, offset, n_frame, n_route, day_slot, C_0=1):
    '''
    Generate data in the form of standard sequence unit.
    :param len_seq: int, the length of target date sequence.  is the number of days
    :param data_seq: np.ndarray, source data / time-series.
    :param offset:  int, the starting index of different dataset type.
    :param n_frame: int, the number of frame within a standard sequence unit,
                         which contains n_his = 12 and n_pred = 9 (3 /15 min, 6 /30 min & 9 /45 min).
    :param n_route: int, the number of routes in the graph.
    :param day_slot: int, the number of time slots per day, controlled by the time window (5 min as default).
    :param C_0: int, the size of input channel.
    :return: np.ndarray, [len_seq, n_frame, n_route, C_0].
    '''
    # in this example time step is 5 min so a day which is 24 hours will be 288 time steps (day_slot=288) 
    # and n_frame is 21 so we will have 288 - 21 + 1 = 268 (n_slot=268)
    n_slot = day_slot - n_frame + 1

    tmp_seq = torch.zeros((len_seq * n_slot, C_0, n_frame, n_route))
    for i in range(len_seq):
        for j in range(n_slot):
            sta = (i + offset) * day_slot + j
            end = sta + n_frame
            tmp_seq[i * n_slot + j, :, :, :] = torch.reshape(data_seq[sta:end, :], [C_0, n_frame, n_route])
    return tmp_seq

In [15]:
def z_score(x, mean, std):
    '''
    Z-score normalization function: $z = (X - \mu) / \sigma $,
    where z is the z-score, X is the value of the element,
    $\mu$ is the population mean, and $\sigma$ is the standard deviation.
    :param x: np.ndarray, input array to be normalized.
    :param mean: float, the value of mean.
    :param std: float, the value of standard deviation.
    :return: np.ndarray, z-score normalized array.
    '''
    return (x - mean) / std

In [16]:
class Datasett(object):
    def __init__(self, data, stats):
        self.__data = data
        self.mean = stats['mean']
        self.std = stats['std']

    def get_data(self, type):
        return self.__data[type]

    def get_stats(self):
        return {'mean': self.mean, 'std': self.std}

    def get_len(self, type):
        return len(self.__data[type])

    def z_inverse(self, type):
        return self.__data[type] * self.std + self.mean

In [17]:
def data_gen(data_seq, data_config, n_route, n_frame=21, day_slot=288):
    '''
    Source file load and dataset generation.
    :param file_path: str, the file path of data source.
    :param data_config: tuple, the configs of dataset in train, validation, test.
    :param n_route: int, the number of routes in the graph.
    :param n_frame: int, the number of frame within a standard sequence unit,
                         which contains n_his = 12 and n_pred = 9 (3 /15 min, 6 /30 min & 9 /45 min).
    :param day_slot: int, the number of time slots per day, controlled by the time window (5 min as default).
    :return: dict, dataset that contains training, validation and test with stats.
    '''
    n_train, n_val, n_test = data_config
    # generate training, validation and test data
#     data_seq = data.values

    seq_train = seq_gen(n_train, data_seq, 0, n_frame, n_route, day_slot)
    seq_val = seq_gen(n_val, data_seq, n_train, n_frame, n_route, day_slot)
    seq_test = seq_gen(n_test, data_seq, n_train + n_val, n_frame, n_route, day_slot)

    # x_stats: dict, the stats for the train dataset, including the value of mean and standard deviation.
    x_stats = {'mean': torch.mean(seq_train), 'std': torch.std(seq_train)}
    print(x_stats)

    # x_train, x_val, x_test: np.array, [sample_size, n_frame, n_route, channel_size].
    x_train = z_score(seq_train, x_stats['mean'], x_stats['std'])
    x_val = z_score(seq_val, x_stats['mean'], x_stats['std'])
    x_test = z_score(seq_test, x_stats['mean'], x_stats['std'])

    x_data = {'train': x_train, 'val': x_val, 'test': x_test}
    dataset = Datasett(x_data, x_stats)
    return dataset

In [18]:
PeMS = data_gen(data_file_tensor, (n_train, n_val, n_test), n_route, n_hist + n_pred)

{'mean': tensor(58.4998), 'std': tensor(13.7286)}


In [19]:
def temporal_conv_layer(x, Kt, c_in, c_out, act_func='relu'):
    '''
    Temporal convolution layer.
    :param x: tensor, [batch_size, time_step, n_route, c_in].
    :param Kt: int, kernel size of temporal convolution.
    :param c_in: int, size of input channel.
    :param c_out: int, size of output channel.
    :param act_func: str, activation function.
    :return: tensor, [batch_size, time_step-Kt+1, n_route, c_out].
    '''

    _, _, T, n = x.size()

    if c_in < c_out:
        # if the size of input channel is less than the output,
        # padding x to the same size of output channel.
        # Note, _.get_shape() cannot convert a partially known TensorShape to a Tensor.
        x_input = torch.cat([x, torch.zeros([x.size()[0], c_out - c_in, T, n])], axis=1)
    else:
        x_input = x
        
    # keep the original input for residual connection.
    x_input = x_input[:, :, Kt - 1:T, :]
        
    if act_func == 'GLU':
        # gated liner unit
    #   tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(wt))
    #   is it really the same as padding= 'valid'
        x_conv = torch.nn.Conv2d(c_in, 2 * c_out, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])(x)
        
        p = (x_conv[:, 0:c_out, :, :] + x_input)
        q = x_conv[:, -c_out:, :, :]

        return p * torch.nn.Sigmoid()(q)
    else:
        x_conv = torch.nn.Conv2d(c_in, c_out, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])(x)
#         tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(wt))
        if act_func == 'linear':
            return x_conv
        elif act_func == 'sigmoid':
            return torch.nn.Sigmoid()(x_conv)
        elif act_func == 'relu':
            return tf.nn.relu(x_conv + x_input)
        else:
            raise ValueError(f'ERROR: activation function "{act_func}" is not defined.')        

In [20]:
def gconv(x, theta, Ks, c_in, c_out, Lk):
    '''
    Spectral-based graph convolution function.
    :param x: tensor, [batch_size, n_route, c_in].
    :param theta: tensor, [Ks*c_in, c_out], trainable kernel parameters.
    :param Ks: int, kernel size of graph convolution.
    :param c_in: int, size of input channel.
    :param c_out: int, size of output channel.
    :return: tensor, [batch_size, n_route, c_out].
    '''
    # graph kernel: tensor, [n_route, Ks*n_route]
    kernel = Lk[0]
    n = tf.shape(kernel)[0]
    # x -> [batch_size, c_in, n_route] -> [batch_size*c_in, n_route]
    x_tmp = tf.reshape(tf.transpose(x, [0, 2, 1]), [-1, n])
    # x_mul = x_tmp * ker -> [batch_size*c_in, Ks*n_route] -> [batch_size, c_in, Ks, n_route]
    x_mul = tf.reshape(tf.matmul(x_tmp, kernel), [-1, c_in, Ks, n])
    # x_ker -> [batch_size, n_route, c_in, K_s] -> [batch_size*n_route, c_in*Ks]
    x_ker = tf.reshape(tf.transpose(x_mul, [0, 3, 1, 2]), [-1, c_in * Ks])
    # x_gconv -> [batch_size*n_route, c_out] -> [batch_size, n_route, c_out]
    x_gconv = tf.reshape(tf.matmul(x_ker, theta), [-1, n, c_out])
    return x_gconv

In [21]:
def spatio_conv_layer(x, Ks, c_in, c_out):
    '''
    Spatial graph convolution layer.
    :param x: tensor, [batch_size, time_step, n_route, c_in].
    :param Ks: int, kernel size of spatial convolution.
    :param c_in: int, size of input channel.
    :param c_out: int, size of output channel.
    :return: tensor, [batch_size, time_step, n_route, c_out].
    '''
    _, _, T, n = x.size()

    if c_in > c_out:
        print("delete me")
#         # bottleneck down-sampling
#         w_input = tf.get_variable('ws_input', shape=[1, 1, c_in, c_out], dtype=tf.float32)
#         tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(w_input))
#         x_input = tf.nn.conv2d(x, w_input, strides=[1, 1, 1, 1], padding='SAME')
#     elif c_in < c_out:
#         # if the size of input channel is less than the output,
#         # padding x to the same size of output channel.
#         # Note, _.get_shape() cannot convert a partially known TensorShape to a Tensor.
#         x_input = tf.concat([x, tf.zeros([tf.shape(x)[0], T, n, c_out - c_in])], axis=3)
    else:
        x_input = x

    ws = torch.rand((Ks * c_in, c_out))
#     tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(ws))
#     variable_summaries(ws, 'theta')
    bs = torch.zeros((c_out))
    # x -> [batch_size*time_step, n_route, c_in] -> [batch_size*time_step, n_route, c_out]
    x_gconv = gconv(torch.reshape(x, [-1, n, c_in]), ws, Ks, c_in, c_out) + bs
    # x_g -> [batch_size, time_step, n_route, c_out]
    x_gc = tf.reshape(x_gconv, [-1, T, n, c_out])
    return tf.nn.relu(x_gc[:, :, :, 0:c_out] + x_input)

In [22]:
def st_conv_block(x, Ks, Kt, channels, scope, rate, act_func='GLU'):
    '''
    Spatio-temporal convolutional block, which contains two temporal gated convolution layers
    and one spatial graph convolution layer in the middle.
    :param x: tensor, batch_size, time_step, n_route, c_in].
    :param Ks: int, kernel size of spatial convolution.
    :param Kt: int, kernel size of temporal convolution.
    :param channels: list, channel configs of a single st_conv block.
    :param scope: str, variable scope.
    :param keep_prob: placeholder, prob of dropout.
    :param act_func: str, activation function.
    :return: tensor, [batch_size, time_step, n_route, c_out].
    '''
    c_si, c_t, c_oo = channels

    x_s = temporal_conv_layer(x, Kt, c_si, c_t, act_func=act_func)
    x_t = spatio_conv_layer(x_s, Ks, c_t, c_t)
#     x_o = temporal_conv_layer(x_t, Kt, c_t, c_oo)
#     x_ln = layer_norm(x_o, f'layer_norm_{scope}')
    
#     return torch.nn.Dropout(x_ln, keep_prob)
    return torch.nn.Dropout(p=rate)(x_s)

In [23]:
def fully_con_layer(x, n, channel):
    '''
    Fully connected layer: maps multi-channels to one.
    :param x: tensor, [batch_size, channel, 1, n_route].
    :param n: int, number of route / size of graph.
    :param channel: channel size of input x.
    :return: tensor, [batch_size, 1, n_route, 1].
    '''
#     tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(w))
#     PyTorch does not support same padding the way Keras does
#     padding='SAME'
    return torch.nn.Conv2d(channel, 1, (1, 1), stride=[1, 1], dilation=[1, 1])(x)

In [24]:
def layer_norm(x):
    '''
    Layer normalization function.
    :param x: tensor, [batch_size, channel, time_step, n_route].
    :return: tensor, [batch_size, channel, time_step, n_route].
    '''
    _, C, _, N = x.size()
    mu, sigma = torch.mean(x, dim=[2, 3], keepdim=True), torch.var(x, dim=[2, 3], keepdim=True)

    gamma = torch.ones([1, C, 1, N])
    beta = torch.zeros([1, C, 1, N])
    _x = (x - mu) / torch.sqrt(sigma + 1e-6) * gamma + beta
    
    return _x

In [25]:
def output_layer(x, T, act_func='GLU'):
    '''
    Output layer: temporal convolution layers attach with one fully connected layer,
    which map outputs of the last st_conv block to a single-step prediction.
    :param x: tensor, [batch_size, channel, time_step, n_route].
    :param T: int, kernel size of temporal convolution.
    :param scope: str, variable scope.
    :param act_func: str, activation function.
    :return: tensor, [batch_size, 1, n_route, 1].
    '''
    _, channel, _, n = x.size()

    # maps multi-steps to one.
    x_i = temporal_conv_layer(x, T, channel, channel, act_func=act_func)
    x_ln = layer_norm(x_i)
    
    x_o = temporal_conv_layer(x_ln, 1, channel, channel, act_func='sigmoid')
    # maps multi-channels to one.
    x_fc = fully_con_layer(x_o, n, channel)
    return x_fc


In [26]:
def build_model(inputs, n_his, Ks, Kt, blocks, rate):
    '''
    Build the base model.
    :param inputs: placeholder.
    :param n_his: int, size of historical records for training.
    :param Ks: int, kernel size of spatial convolution.
    :param Kt: int, kernel size of temporal convolution.
    :param blocks: list, channel configs of st_conv blocks.
    :param keep_prob: placeholder.
    '''
    x = inputs[:, 0:n_his, :, :]

    # Ko>0: kernel size of temporal convolution in the output layer.
#     Ko = n_his
    Ko = 18
    # ST-Block
    for i, channels in enumerate(blocks):
        x = st_conv_block(x, Ks, Kt, channels, i, rate, act_func='GLU')
#         Ko -= 2 * (Ks - 1)

    # Output Layer
    if Ko > 1:
        y = output_layer(x, Ko)
    else:
        raise ValueError(f'ERROR: kernel size Ko must be greater than 1, but received "{Ko}".')

#     tf.add_to_collection(name='copy_loss',
#                          value=tf.nn.l2_loss(inputs[:, n_his - 1:n_his, :, :] - inputs[:, n_his:n_his + 1, :, :]))
    loss_fn = torch.nn.MSELoss()
    
#     y size
#     torch.Size([50, 1, 19, 228])
#     input
#     torch.Size([50, 0, 21, 228])


    train_loss = loss_fn(y, inputs[:, :, n_his:n_his + 1, :])
    single_pred = y[:, 0, :, :]
#     tf.add_to_collection(name='y_pred', value=single_pred)
    return train_loss, single_pred

In [27]:
def model_train(inputs, blocks, n_route, n_hist, n_pred, Ks, Kt, batch_size, epoch, inf_mode, opt, rate, sum_path='./output/tensorboard'):
    '''
    Train the base model.
    :param inputs: instance of class Dataset, data source for training.
    :param blocks: list, channel configs of st_conv blocks.
    '''
    
    # Placeholder for model training
#     x = (tf.float32, [None, n_his + 1, n, 1], name='data_input')

    writer = SummaryWriter()

    len_train = inputs.get_len('train')
    if len_train % batch_size == 0:
        epoch_step = len_train / batch_size
    else:
        epoch_step = int(len_train / batch_size) + 1
        
    writer.add_scalar('learning_rate', lr)
#     step_op = tf.assign_add(global_steps, 1)
    
#     if opt == torch.optim.RMSprop:
#         train_op = torch.optim.RMSprop(lr)
#     elif opt == 'ADAM':
#         train_op = torch.optim.Adam(lr, weight_decay=0.01).minimize(train_loss)
#     else:
#         raise ValueError(f'ERROR: optimizer "{opt}" is not defined.')

#     merged = tf.summary.merge_all()

#     writer = tf.summary.FileWriter(pjoin(sum_path, 'train'), sess.graph)
    if inf_mode == 'merge':
        # for inference mode 'merge', the type of step index is np.ndarray.
        step_idx = tmp_idx = np.arange(3, n_pred + 1, 3) - 1
        min_val = min_va_val = np.array([4e1, 1e5, 1e5] * len(step_idx))
    else:
        raise ValueError(f'ERROR: test mode "{inf_mode}" is not defined.')

#     train_op.zero_grad()
    for i in range(epoch):
        start_time = time.time()
        
        data_loader = DataLoader(inputs.get_data('train'),
                     batch_size=batch_size,
                     drop_last=True)
# gen_batch(inputs.get_data('train'), batch_size, dynamic_batch=True, shuffle=True)

        j = 0
        for x_batch in iter(data_loader):
            x = x_batch[:, 0:n_hist + 1, :, :]
            rate = 0
            
            # Define model loss
            train_loss, pred = build_model(x, n_hist, Ks, Kt, blocks, rate)
            writer.add_scalar('train_loss', train_loss)
#             copy_loss = tf.add_n(tf.get_collection('copy_loss'))
#             writer.add_scalar('copy_loss', copy_loss)
    
#             summary, _ = sess.run([merged, train_op])
#             writer.add_summary(summary, i * epoch_step + j)
#             if j % 50 == 0:
#                 loss_value = sess.run([train_loss, copy_loss]
#                 print(f'Epoch {i:2d}, Step {j:3d}: [{loss_value[0]:.3f}, {loss_value[1]:.3f}]')
            j += 1
        print(f'Epoch {i:2d} Training Time {time.time() - start_time:.3f}s')

        start_time = time.time()
        min_va_val, min_val = \
                model_inference(sess, pred, inputs, batch_size, n_his, n_pred, step_idx, min_va_val, min_val)

            
        loss = loss_fn(predictions, t)
        loss.backward()
#         train_op.step()
        for ix in tmp_idx:
            va, te = min_va_val[ix - 2:ix + 1], min_val[ix - 2:ix + 1]
            print(f'Time Step {ix + 1}: '
                    f'MAPE {va[0]:7.3%}, {te[0]:7.3%}; '
                    f'MAE  {va[1]:4.3f}, {te[1]:4.3f}; '
                    f'RMSE {va[2]:6.3f}, {te[2]:6.3f}.')
        print(f'Epoch {i:2d} Inference Time {time.time() - start_time:.3f}s')

#         if (i + 1) % args.save == 0:
#             model_save(sess, global_steps, 'STGCN')
    writer.close()
    print('Training model finished!')

In [28]:

model_train(PeMS, blocks, n_route, n_hist, n_pred, ks, kt, batch_size, epoch, inf_mode, opt, rate)

x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])


/home/raha/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([50, 1, 1, 228])) that is different to the input size (torch.Size([50, 1, 2, 228])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
tor

x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torc

x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torc

x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
to

x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
torch.Size([50, 1, 1, 228])
x before conv 2d
torch.Size([50, 1, 21, 228])
x_conv
torch.Size([50, 32, 19, 228])
x_input before residual
torch.Size([50, 32, 19, 228])
x before conv 2d
torch.Size([50, 32, 19, 228])
x_conv
torch.Size([50, 32, 2, 228])
x_input before residual
torch.Size([50, 32, 2, 228])
y size
torch.Size([50, 1, 2, 228])
input
to

KeyboardInterrupt: 